In [ ]:
# WebScraping

In [7]:
# Librerías 

import pandas as pd
import numpy as np
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import math
import time
from datetime import datetime
from selenium.webdriver.common.keys import Keys

In [8]:
#Driver de chrome 
from selenium.webdriver.chrome.service import Service
chrome_driver = "chromedriver.exe"
driver_service = Service(executable_path = chrome_driver)
## EL DRIVER DEBE ESTAR EN LA MISMA CARPETA QUE EL ARCHIVO

In [9]:
navegador = webdriver.Chrome(service=driver_service) ##Abrimos el chrome del Selenium IMPORTANTE, QUITAR LOS POP UP DEL WALLAPOP AL PRINCIPIO

In [10]:
# Esta función coge los links de aquellos anuncios de los que queremos obtener infomación.
# Filtra por precio de coche y se va moviendo en rangos de 1000

def busca_coches(precio_minimo,num_coches):

    links = [] #Donde se almacenaran todos los links
    print("coches de",precio_minimo,"a",precio_minimo+1000) #Chivato para saber por donde vamos
    url = "https://es.wallapop.com/app/search?min_sale_price="+str(precio_minimo)+"&max_sale_price="+str((precio_minimo+1000))+"&filters_source=default_filters&category_ids=100&longitude=-3.69196&latitude=40.41956"
    navegador.get(url) ##Cogemos la url
    time.sleep(3) ## Esperamos un poco a que cargue, por si acaso
    navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    boton = navegador.find_element(By.ID, "btn-load-more") ## El boton de cargar mas productos. AVECES DA ERROR PORQUE DI
    boton.click() ##Pulsamos el boton AVECES DA ERROR DEJAR EL NAVEGADOR ABIERTO COMPLETAMENTE PARA QUE NO OCURRA
    time.sleep(2) ## El boton parece que tarda bastante en cargar, por eso puse un timer de 5 segundos, seguro que se podrá bajar
        
    for i in range(round((num_coches)/40)): #Cada iteracion corresponde a 40 fichas de coches más, a partir de 80 interacciones ya comienza a ir lento en mi pc
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);") ## Hacemos scroll tantas veces como digamos en el for
        time.sleep(1)
        print("Cargando",i) ## Para saber que funciona
            
    navegador.page_source ## Necesitamos el soup de la página con todo el scroll
    soup = BeautifulSoup(navegador.page_source) #Soup de la pagina cargada
    time.sleep(2) #Que cargue todo
    ficha = soup.find_all("a", attrs = {"class":"ItemCardList__item"}) #Encuentra la ficha de todos los coches
        
    for i in range(len(ficha)): #Para cada ficha de coche...
        links.append(ficha[i]["href"]) #Añade links a la lista
    print(len(ficha),"Links metidos") #Chivato, nos dira cuantos links de coches ha introducido

    #Una vez tenemos todos los links ejecutamos la funcion que recopila la informacion de cada uno de ellos
    #data_pagina = sacar_datos(links) 
    return links #Devolverá los datos 

In [11]:
# Esta función entra en cada link y extrae la información, montando un DataFrame

def sacar_datos(links):
    
        
    precios=[]
    marcas=[]
    modelos=[]
    year=[]
    kms=[]
    type_vehi=[]
    plazas=[]
    puertas=[]
    type_comb=[]
    potencias=[]
    type_cambio=[]
    texto=[]
    nombre_usuario=[]
    estrellas=[]
    resenas=[]
    ubicacion=[]
    num_ventas=[]
    
    for j in links:
        print(j)
        navegador.get(j)
        time.sleep(1)
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)
        soup = BeautifulSoup(navegador.page_source)
        error = soup.find('div', attrs = {'class': 'not-found-page_Error__container__GtFkg d-flex flex-column align-items-center'}) #Para cuando la pagina no exista
        error2 = soup.find('div', attrs = {'class': 'error-page_Error__container__qaYIn d-flex flex-column align-items-center'}) ## Para cuando la pagina de error
        if error != None: ## Si la pagina no existe el contenido del error no estará vacio
            continue ## Pasaremos a la siguiente iteracion, ya que en la página vacia no hay datos que poner       
        if error2 != None: ## Para otro problema en la pagina
            continue

            
        precio=soup.find('div', attrs={'aria-label':'Item Price'})
        if len(precio)==1:
            precios.append(precio.span.text.split()[0])
        elif len(precio)==2:
            p=precio.find('span', attrs={'class':'item-detail-price_ItemDetailPrice--standardFinanced__14D3z'})
            precios.append(p.text.split()[0])

        else:
            precios.append(np.NaN)
      

    # Vamos a sacar el nombre de usuario
        nombre = soup.find_all('h3')
        
        # nombre = soup.find('h3', attrs={'class':'text-truncate mb-0 item-detail-header_ItemDetailHeader__text--typoMidM__VeCLc'})
        if nombre is not None:
            nombre_usuario.append(nombre[0].text)
        else:
            nombre_usuario.append(np.NaN)

    # Vamos a sacar la lista de comentarios. El if/else es para cuando alguno nos de un problema.
        textos = soup.find("section", attrs = {"class":"item-detail_ItemDetail__description__7rXXT py-4"})
        if textos is not None:
            texto.append(textos.text)
        else:
            texto.append(np.NaN)

        # precio=soup.find('div', attrs={'aria-label':'Item Price'})
        # precios.append(precio.span.text.split()[0])
        # marc_mod=soup.find_all('span', attrs={'class':"item-detail-seo-bubble_ItemDetailSEOBubble__link__yZP8O px-3 py-2 text-truncate align-middle"})
        # marcas.append(marc_mod[0].text)
        # modelo.append(marc_mod[1].text)

    # Estrellas
        stars = soup.find("span", attrs = {"class":"item-detail-header_ItemDetailHeader__text--typoLowS__9JNQi ml-1"})
        if stars != None:
            estrellas.append(stars.text)
        else:
            estrellas.append(np.NaN)

    # Reseñas 
        num_resenas = soup.find("span", attrs = {"class":"item-detail-header_ItemDetailHeader__text--typoLowSContentLow__UyhmS ml-1"})
        if num_resenas != None:
            resenas.append(num_resenas.text[1])
        else:
            resenas.append(np.NaN)
    # Nº de ventas
        num_venta = soup.find('span', attrs = {'class': 'item-detail-header_ItemDetailHeader__text--typoLowS__9JNQi mr-1'})
        if num_venta != None:
            num_ventas.append(num_venta.text.split()[0])
        else:
            num_ventas.append(np.NaN)
    # Ubicación
        ubicaciones = soup.find("a", attrs = {"class":"item-detail-location_ItemDetailLocation__link__s4oPx"})
        if ubicaciones == None: ## Excepcion, a veces da error
            ubicaciones = soup.find("div", attrs = {"class":"d-flex item-detail-location_ItemDetailLocation___QiCU"})
            if ubicaciones == None: ##En casa de que tampoco hubiera ubicacion
                continue ##Pase al siguiente
        ubicacion.append(ubicaciones.text)
    # Vamos a sacar la lista de características (tipo, nº de plazas, nº de puertas, , tipo de combustible, potencia en CV, tipo de cambio).
    # El if/else es para cuando alguno nos de un problema.
        carac=soup.find_all('span', attrs={'class':"item-detail-attributes-info_AttributesInfo__measure__uZS62 mt-1"})
        if len(carac)==6:
            type_vehi.append(carac[0].text)
            plazas.append(carac[1].text.split()[0])
            puertas.append(carac[2].text.split()[0])
            type_comb.append(carac[3].text)
            potencias.append(carac[4].text.split()[0])
            type_cambio.append(carac[5].text)
        else:
            type_vehi.append(np.NaN)
            plazas.append(np.NaN)
            puertas.append(np.NaN)
            type_comb.append(np.NaN)
            potencias.append(np.NaN)
            type_cambio.append(np.NaN)


    # Vamos a sacar la lista de varios (marca, modelo, año y kilómetros).
    # El if/else es para cuando alguno nos de un problema.
        varios=soup.find_all('div',attrs={'class':"item-detail-car-extra-info_ItemDetailCarExtraInfo__section__n4g_P d-flex align-items-center"})
        if len(varios)==4:
            marcas.append(varios[0].find_all('span')[1].text)
            modelos.append(varios[1].find_all('span')[1].text)
            year.append(varios[2].find_all('span')[1].text)
            kms.append(varios[3].find_all('span')[1].text)

        else:
            marcas.append(np.NaN)
            modelos.append(np.NaN)
            year.append(np.NaN)
            kms.append(np.NaN)

In [12]:
#Creacion del dataframe donde meteremos todos los datos. Recordar  vaciarlo 
data_total = pd.DataFrame([], columns=['Marca', 'Modelo', 'Año', 'Potencia', 'Plazas', 'Nº de puertas',
         'km', 'Tipo de vehiculo', 'Combustible', 'Cambio', 'Comentarios','Nombre usuario',
          'Puntuacion', 'Nº valoraciones', 'Nº Ventas', 'Ubicacion', 'Precio' ])

In [14]:
#Cada iteración irá del precio minimo al precio minimo +1000, para abarcar el máximo número de coches posibles
# A partir de 4000 coches en num_coches el rendimiento del navegador empieza a ir lento (al menos el mio)
precio_minimo = 8000 ##Empezamos a buscar coches a partir del rango mínimo de precio (1000) no he puesto 0 porque salen ventas raras
precio_maximo = 10000 ##Precio máximo hasta donde queremos buscar. NO PUEDE SER IGUAL AL PRECIO MINIMO
num_coches = 3000 # Numero de coches que queremos coger por cada rango de precio. Es una medida APROXIMADA, siempre suele coger 100+ de lo indicado, irrelevante, queremos MUCHOS datos
links = [] ## Para la condicion del while
inicio = 0 ## Por que numero de link empezara a sacar datos. Por defecto el 0. 

for j in range((precio_maximo-precio_minimo)//1000):
    links = []
    contador = 0 ##Para que no se quede en bucle infinito si no encontrara los links, podría darse el caso de que no hubiera tal cantidad de links
    while len(links) < (num_coches-(num_coches/10)): ##Para cuando nos coge menos de 3000 links. Le dejo un margen de un 10% 3000 = 2700 - 3000        
        print("metemos coches de",precio_minimo)
        links = busca_coches(precio_minimo,num_coches) ## Sacame los datos de la pagina con el filtro aplicado, entre el rango de precios indicado arriba
        contador+=1 #Para romper el bucle en caso de que no encontrara el número de links que queremos. Le damos 3 intentos para buscarlo
        if contador > 3:
            break

    for j in range(inicio,len(links)+1):
        print("Link nº",j) #Así veriamos donde se ha dado el error
        dato_link = sacar_datos(links[j-1:j]) ## Que llame a la funcion tantas veces como numero de links tengamos
        data_total = pd.concat([data_total, dato_link], ignore_index=True) # No se como añadir el dataframe de dato_link sin concatenarlo
    
    data_total.to_csv(str(precio_minimo)+"CochesWallapop_Mario.csv",index = False)
    print("datos entre",str(precio_minimo),"y",str(precio_minimo+1000),"introducidos")
    precio_minimo+=1000 ## Pasamos al siguiente rango
    print(precio_minimo, "que pasaaaaaa que no suma")
    time.sleep(60) #Vamos a descansar
    
    data_total = pd.DataFrame([], columns=['Marca', 'Modelo', 'Año', 'Potencia', 'Plazas', 'Nº de puertas',
         'km', 'Tipo de vehiculo', 'Combustible', 'Cambio', 'Comentarios','Nombre usuario',
          'Puntuacion', 'Nº valoraciones', 'Nº Ventas', 'Ubicacion', 'Precio' ])



metemos coches de 8000
coches de 8000 a 9000


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.72)
Stacktrace:
	GetHandleVerifier [0x00007FF6AFFBEEA2+31554]
	(No symbol) [0x00007FF6AFF37ED9]
	(No symbol) [0x00007FF6AFDF872A]
	(No symbol) [0x00007FF6AFDCD995]
	(No symbol) [0x00007FF6AFE744D7]
	(No symbol) [0x00007FF6AFE8C051]
	(No symbol) [0x00007FF6AFE6CDD3]
	(No symbol) [0x00007FF6AFE3A33B]
	(No symbol) [0x00007FF6AFE3AED1]
	GetHandleVerifier [0x00007FF6B02C8B1D+3217341]
	GetHandleVerifier [0x00007FF6B0315AE3+3532675]
	GetHandleVerifier [0x00007FF6B030B0E0+3489152]
	GetHandleVerifier [0x00007FF6B006E776+750614]
	(No symbol) [0x00007FF6AFF4375F]
	(No symbol) [0x00007FF6AFF3EB14]
	(No symbol) [0x00007FF6AFF3ECA2]
	(No symbol) [0x00007FF6AFF2E16F]
	BaseThreadInitThunk [0x00007FFA324F7374+20]
	RtlUserThreadStart [0x00007FFA333DCC91+33]


In [10]:
# Si buscas varios rangos de 1000 (ej. de 5000 a 10000) y da algún fallo o quieres interrumpirlo manualmente,
# puedes guardar a mano los datos que tengas en un csv, siempre que haya completado algún rango de 1000
# ejemplo: busco de 5000 a 10000 y cuando está buscando de 7000 a 8000 da fallo, o interrumpo manualmente; podría guardar a mano los
# rangos de 5000 a 6000 y de 6000 a 7000
data_total.to_csv('Wallapop_9-10.csv', index=False)

In [ ]:
# Una vez tengamos el archivo .csv con los datos sucios, hay que limpiarlos con las funciones 'limpieza_datos' y 'agrupa marcas'
# Importante cargar el .csv de la forma adecuada

In [ ]:
import pandas as pd
from unidecode import unidecode

In [ ]:
# Esta es la función general de limpieza: cambia caracteres y sirve de filtro. 
# Para que funcione correctamente, el archivo .csv obtenido del web scrapping debe cargarse de la siguiente manera:
# data1 = pd.read_csv('Wallapop_varios.csv', dtype={'Marca': 'object', 'Modelo': 'object', 'Año': 'float64', 'Potencia': 'object',
#                                              'Plazas': 'float64', 'Nº de puertas': 'float64', 'km': 'float64', 'Tipo de vehiculo': 'object',
#                                              'Combustible': 'object', 'Cambio': 'object', 'Comentarios': 'object', 'Nombre usuario': 'object',
#                                              'Puntuacion': 'float64', 'Nº valoraciones': 'float64', 'Nº ventas': 'float64', 
#                                              'Ubicacion': 'object', 'Precio': 'object'})

In [ ]:
def limpieza_datos(df):

## Quitamos nulos y duplicados
    df = df.dropna()
    df = df.drop_duplicates()
    
    ## Procedemos con las variables:
    
    #Año
    df = df[df["Año"] >=1900] ## Quitamos todos los coches con año inferior a 1900
    df["Año"] = df["Año"].astype("int") ## Pasamos todo a int
    
    #Potencia   
    df.loc[df['Potencia'].str.len()>5, 'Potencia'] = "10000" ##Para cuando el string sea mayor a 5, que serán potencias con errores como "caballos" o 1,324,54    
    df['Potencia'] = df['Potencia'].str.replace(',', '', regex=False) # Si hay comas que las quite, que luego da problemas
    df['Potencia'] = df["Potencia"].astype(float)
    df['Potencia'] = round(df["Potencia"])
    df['Potencia'] = df["Potencia"].astype(int)
    df = df[(df["Potencia"] <=500) & (df["Potencia"] >=40)] ##Quitamos coches con potencia superior a 500

    #Puertas
    df["Nº de puertas"] = df["Nº de puertas"].astype("float")
    df['Nº de puertas'] = round(df["Nº de puertas"])
    df["Nº de puertas"] = df["Nº de puertas"].astype("int") ## Pasamos todo a int
    df = df[(df['Nº de puertas'] >= 2) & (df['Nº de puertas'] <= 10)] ##Quitamos menor a 2 puertas y superiores a 10
    
    #Kilometros
    df["km"] = df["km"].astype("int") #Pasamos a int
    df = df[df["km"] <=1000000] ##Coches con mas de 1m de km fuera
    
    #Plazas
    df["Plazas"] = df["Plazas"].astype("float")
    df['Plazas'] = round(df["Plazas"])
    df["Plazas"] = df["Plazas"].astype("int") ##Pasamos a int
    df = df[(df['Plazas'] >= 2) & (df['Plazas'] <= 10)] ##Quitamos menor a 2 plazas y superiores a 10
    
    #Puntuaciones y ventas las pasamos a int
    df["Puntuacion"] = df["Puntuacion"].astype("float")
    df['Puntuacion'] = round(df["Puntuacion"])
    df["Puntuacion"] = df["Puntuacion"].astype("int") ## Pasamos todo a int
    df["Nº Ventas"] = df["Nº Ventas"].astype("int") ## Pasamos todo a int
    
    #Eliminamos nº de valoraciones ya que no pudieron obtenerse correctamente
    df = df.drop(columns=['Nº valoraciones'])
    
    #Transformaciones a la variable objetivo para pasar de string a int
    df['Precio'] = df['Precio'].str.replace('.', '', regex=False) ##Quitamos puntos que molesten
    df['Precio'] = df['Precio'].str.replace(',', '.', regex=False) # Cambiamos , por . para que pueda pasarlo a float
    df['Precio']=df['Precio'].astype('float') ##Pasamos a float para las comas. Si pasamos directamente a int da error
    df['Precio']=df['Precio'].astype('int') ##De float lo pasamos a int

    df = df.drop_duplicates(subset=['Marca','Modelo','Año','Potencia','Plazas','Nº de puertas',
                                'km','Tipo de vehiculo','Combustible','Cambio','Precio'])
    
    return df

In [ ]:
# Esta función sirve para agrupar las marcas, independientemente de errores ortográficos o distición entre mayúsculas y minúsculas

def agrupa_marcas(data):
    from unidecode import unidecode
    data["Marca"] = data["Marca"].str.upper() ## Primero pasamos todo a mayusculas, ya con esto homogeneizamos columnas
    data["Marca"] = data["Marca"].apply(unidecode) ## Quitamos todos los acentos. Recordar importar el unidecode from unidecode import unidecode
    marcas = data["Marca"].value_counts().index # Metemos todas las marcas de coches que tenemos

    for i in marcas: ##Ahora para cada marca, empezando desde la primera
        contiene_marca = data["Marca"].str.contains(i, case=False, na=False) 
        data.loc[contiene_marca, "Marca"] = i ##Sustituimos la columna que contiene esa marca con la marca inicial

    #Una vez agrupadas las marcas eliminamos aquellas únicas que tengan pocos valores
    marcas_eliminar = []
    filtro = data["Marca"].value_counts() <= 5 #Metemos en el filtro todas las marcas y si tienen menos de 5 filas será true

    for i in range(len(filtro)):
        if filtro.values[i] == True: ##
            marcas_eliminar.append(filtro.index[i])

    data = data[~data["Marca"].isin(marcas_eliminar)] ## Eliminamos todas las filas donde esten las marcas a eliminar, la virgulilla dice que haga lo contrario
    return data